## Testing with flattened CellMap

In [1]:
import os
import numpy as np
import pandas as pd
import config
from source.utils import loadmat
import starfish as sf

In [2]:
# dir_path = os.path.dirname(os.path.realpath(__file__))
roi = config.DEFAULT['roi']
label_image_path = os.path.join('./', 'demo_data', 'CellMap.mat')
label_image = loadmat(os.path.join(label_image_path))
label_image = label_image["CellMap"]

In [3]:
t = label_image>0
# pd.DataFrame(label_image)

In [4]:
df = pd.DataFrame(label_image).unstack()
df

0     0       0
      1       0
      2       0
      3       0
      4       0
             ..
7601  5466    0
      5467    0
      5468    0
      5469    0
      5470    0
Length: 41590542, dtype: uint32

In [5]:
df_2 = df[df>0]
cellmap = df_2.reset_index().rename(columns={'level_0': 'y', 'level_1': 'x', 0: 'parent_cell'})
cellmap.head()

,y,x,parent_cell
0,0,718,1
1,0,719,1
2,0,720,1
3,0,721,1
4,0,722,1


In [6]:
cellmap.x==13001

0          False
1          False
2          False
3          False
4          False
           ...  
3721907    False
3721908    False
3721909    False
3721910    False
3721911    False
Name: x, Length: 3721912, dtype: bool

In [7]:
saFile = os.path.join('./', 'demo_data', 'spots.csv')

In [8]:
sa_df = pd.read_csv(saFile)
sa_df = sa_df.rename(columns={'xc': 'x', 'yc': 'y', 'zc': 'z', 'Gene': 'target'})
sa = sf.core.types.SpotAttributes(sa_df)
sa.data['x'] = sa.data.x - 1
sa.data['y'] = sa.data.y - 1
sa.data['x'] = sa.data['x'].astype(int)
sa.data['y'] = sa.data['y'].astype(int)

In [9]:
sa.data['x'] = sa.data['x'] - 6150
sa.data['y'] = sa.data['y'] - 12987

In [10]:
sa.data.head()

,index,target,radius,x,y,z
0,0,Cxcl14,NaN,110,14,0
1,1,Plp1,NaN,-1,736,0
2,2,Plp1,NaN,-1,746,0
3,3,Id2,NaN,-1,993,0
4,4,Enpp2,NaN,-1,1016,0


In [11]:
merged = pd.merge(sa.data, cellmap, on=['x','y'], how='left')
merged.head()

,index,target,radius,x,y,z,parent_cell
0,0,Cxcl14,NaN,110,14,0,NaN
1,1,Plp1,NaN,-1,736,0,NaN
2,2,Plp1,NaN,-1,746,0,NaN
3,3,Id2,NaN,-1,993,0,NaN
4,4,Enpp2,NaN,-1,1016,0,NaN


In [12]:
merged[np.isfinite(merged.parent_cell)]

,index,target,radius,x,y,z,parent_cell
364,364,Plp1,NaN,84,187,0,83.0
373,373,Plp1,NaN,86,175,0,83.0
387,387,Plp1,NaN,89,178,0,83.0
399,399,Enpp2,NaN,92,178,0,83.0
406,406,Enpp2,NaN,93,184,0,83.0
438,438,Plp1,NaN,98,189,0,83.0
489,489,Plp1,NaN,104,173,0,83.0
529,529,Plp1,NaN,108,184,0,83.0
532,532,Cryab,NaN,109,177,0,83.0
539,539,Plp1,NaN,110,171,0,83.0


In [13]:
np.isfinite(merged.parent_cell).sum()

4574